# From xdf to csv's

This script simply loops over your ./Data folder's subfolders (containing participant level data). It looks for .xdf files, and then writes the time vector and channel data to a csv time series.

In [1]:
import os
import pyxdf
import glob
import pandas as pd
import numpy as np
import wave, struct, math, random
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip #this is the video clipping function
from moviepy.video.io.VideoFileClip import VideoFileClip #alternative for snipping video
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
#identify xdf files in data folder
    #Note this data folder is external to the GITHUB page, because they are enormous files.
    #So just make sure you have a data folder called VENI_data_local
    
datafolder = 'F:/exp_100_data/'
errorlogs = datafolder+'/error_logs/'
experiment_to_process = 'confirmatory_data_raw/'
#outputfolder raw
outputfolder = datafolder+'Data_processed_2/CsvDataTS_raw/'
#outputfolder trialbased
trialfolder = datafolder+'Data_processed_2/Data_trials/'
timejson = datafolder+'Data_processed_2/time_shift_json/'

targetfolder = datafolder+experiment_to_process
datafolders = os.listdir(targetfolder)


# Identifying files and folders

In [2]:
#lets identify all xdf files and all the associated triallist info
xdffiles = []
trialdatas = []
for i in datafolders:
    #absolute path to search all text files inside a specific folder
    file = glob.glob(targetfolder+i+'/*.xdf')
    trialfile = targetfolder+i+'/triallist_'+i+'.csv'
    trialdatas.append(trialfile)
    xdffiles.extend(file)
    
#audio write
def to_audio(fileloc, timeseries, samplerate = 16000, channels = 1):
    if timeseriestype == 'Mic': 
        obj = wave.open(fileloc,'w')
        obj.setnchannels(channels) # mono
        obj.setsampwidth(2)
        obj.setframerate(float(samplerate))
        for i in timeseries:
            data = struct.pack('<h', int(i[0]))
            obj.writeframesraw( data )
        obj.close()
print(xdffiles)
print(trialdatas)


#for dat in datafolders:
#    print('Loading in data from participant: ' + dat)
#    #absolute path to search all text files inside a specific folder
#    files = glob.glob(targetfolder+dat+'/*.xdf')
#    streams, header = pyxdf.load_xdf(files[0])
#    print(streams)

['F:/exp_100_data/confirmatory_data_raw/p1\\p1.xdf', 'F:/exp_100_data/confirmatory_data_raw/p10\\p10.xdf', 'F:/exp_100_data/confirmatory_data_raw/p11\\p11.xdf', 'F:/exp_100_data/confirmatory_data_raw/p12\\p12.xdf', 'F:/exp_100_data/confirmatory_data_raw/p13\\p13.xdf', 'F:/exp_100_data/confirmatory_data_raw/p14\\p14.xdf', 'F:/exp_100_data/confirmatory_data_raw/p15\\p15.xdf', 'F:/exp_100_data/confirmatory_data_raw/p16\\p16.xdf', 'F:/exp_100_data/confirmatory_data_raw/p17\\p17.xdf', 'F:/exp_100_data/confirmatory_data_raw/p18\\p18.xdf', 'F:/exp_100_data/confirmatory_data_raw/p2\\p2.xdf', 'F:/exp_100_data/confirmatory_data_raw/p3\\p3.xdf', 'F:/exp_100_data/confirmatory_data_raw/p41\\p41.xdf', 'F:/exp_100_data/confirmatory_data_raw/p42\\p42.xdf', 'F:/exp_100_data/confirmatory_data_raw/p5\\p5.xdf', 'F:/exp_100_data/confirmatory_data_raw/p6\\p6.xdf', 'F:/exp_100_data/confirmatory_data_raw/p7\\p7.xdf', 'F:/exp_100_data/confirmatory_data_raw/p8\\p8.xdf', 'F:/exp_100_data/confirmatory_data_raw/p9

# breaking the xdf files single csv files and audio per stream

In [20]:
from scipy.io import wavfile
import noisereduce as nr
import json

errorlist = []

#timeshift function
    #the button box and the button box as registered in psychopy have a shift in clocks
    #we can retrieve this shift by looking for duration in between button presses
    # and match them up in both series, so as to detect a button press in both psychopy and the streams
def calc_timeshift(start_times, tt_bb):
    derivativest = np.round(np.diff(start_times),3)
    derivativest = np.insert(derivativest, 0, 0)
    #take the second element the trialtime and corresponding derivative value
    firstbutton = start_times[1]
    derivativest = derivativest[1]
    #compute the difference
    print(firstbutton)
    print(derivativest)

    #compute the times added over time
    buttonpress_indices = np.where(np.diff(tt_bb[10]==65534.0) != 0)[0] + 1
    buttonprestimes = list(tt_bb[0][buttonpress_indices ])
    derbb =  np.insert(np.diff(buttonprestimes), 0, 0)
    cumsBP = list(np.cumsum(derbb))

    #which element is same as the first time
    indextargettimes = abs(cumsBP-derivativest)
    indextargettimes = np.argmin(indextargettimes)

    timeshift = firstbutton-buttonprestimes[indextargettimes]
    return(timeshift)

time_shifts = {} 

for dat in datafolders:
    print('Loading in data from participant: ' + dat)
    trialdata = pd.read_csv(targetfolder+dat+'/triallist_'+dat+'.csv', sep=";")
    #absolute path to search all text files inside a specific folder
    files = glob.glob(targetfolder+dat+'/*.xdf')
    streams, header = pyxdf.load_xdf(files[0])   
    #############################CALCULATE TIMESHIFT
    for stream in streams:
        timeseriestype = stream['info']['name'][0]
        if timeseriestype == 'BrainAmpSeries-Dev_1':
          #lets calcuate the difference in times for trialstart and button presses to correctly align things again
            #get the data for brainamp
            timevec = stream['time_stamps']
            timeseries = stream['time_series']
            matrix_aux = np.vstack([np.transpose(timevec),np.transpose(timeseries)])
            matrix     = np.transpose(matrix_aux)
            df_lab = pd.DataFrame(matrix)
            #apply time shift calc
            start_times = trialdata['timestamp']
            tt_bb = df_lab
            time_shift = calc_timeshift(start_times, tt_bb)
            print('time shift calcuated: ' + str(time_shift))
            time_shifts[dat] = time_shift
    #we go through each stream and save it as a csv, or additionally as a .wav file
    for stream in streams:
        timeseriestype = stream['info']['name'][0]
        samplerate = round(float(stream['info']['nominal_srate'][0]))
        #in the xdf loop over the streams and save it as csv if not yet exists
        channelcount = stream['info']['channel_count'][0]
        print('working on stream: ' + timeseriestype + '  with a channel count of ' + str(channelcount) +'\n and a sampling rate of ' + str(samplerate))
        timevec = stream['time_stamps']
        timeseries = stream['time_series']
        matrix_aux = np.vstack([np.transpose(timevec),np.transpose(timeseries)])
        matrix     = np.transpose(matrix_aux)
        df_lab = pd.DataFrame(matrix)
        df_lab.to_csv(outputfolder+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.csv',index=False)
        #for audio data also create a wav file
        if timeseriestype == 'Mic':
            wavloc = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.wav'
            to_audio(wavloc, timeseries)
            # load data
            rate, data = wavfile.read(wavloc)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=data, sr=rate, n_std_thresh_stationary=1.5,stationary=True)
            wavloc2 = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_denoised.wav'
            wavfile.write(wavloc2, rate, reduced_noise)
        #############################Loop through trial times (adjusted for timeshift) and snip individual trials
        print('done with processing a complete time series and audio data')
        print('we will now start making trial snipped data') 
        for index, row in trialdata.iterrows():
            begin = row['trialstarted']-time_shift
            end = row['trialended']-time_shift
            trialindex = row['trialindex']            
            indices = (df_lab.loc[:,0] > begin) & (df_lab.loc[:,0] < end)
            beginst = min(df_lab.loc[:,0]) #start time of the timeseries
            endst = max(df_lab.loc[:,0])  #end time of the timeseries
            subset = df_lab.loc[indices, :]
            if(len(subset.axes[0])<2):
                errorlist.append(dat + " for "+ timeseriestype + " for trial " + str(trialindex) + 'NO DATA WITHIN RANGE...')
            if(len(subset.axes[0])>2):
                #save subset to csv
                subset.to_csv(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+timeseriestype+'.csv', index=False)
                if timeseriestype == 'Mic':
                    wavloc = trialfolder+dat+'_trial_'+ str(trialindex) +'_'+timeseriestype+'.wav'
                    to_audio(fileloc = wavloc, timeseries = timeseries[indices])
                    #also apply denoising
                    reduced_noiseclip = reduced_noise[indices]
                    wavloc2 = trialfolder+dat+'_trial_'+ str(trialindex) +'_'+timeseriestype+'denoised.wav'
                    wavfile.write(wavloc2, rate, reduced_noiseclip)            
        #after every participants well save the error log
        errors = pd.DataFrame(errorlist, columns=['file_error'])
        file_path = errorlogs+'error_log_cuttingtrails_may2023.csv'
        errors.to_csv(file_path, index=False)  
        ###################################save the timeshift
        # Convert the dictionary to a JSON string
        json_string = json.dumps(time_shifts)
        # Specify the file path
        file_path = timejson+'timeshift_dictionary.json'
        # Write the JSON string to a file
        with open(file_path, "w") as file:
            file.write(json_string)
        
print('Were done: proceed to snipping videos to triallevel')

Loading in data from participant: p1
2531.6167769
21.837
time shift calcuated: 262.8137029243908
working on stream: BrainAmpSeries-Dev_1  with a channel count of 10
 and a sampling rate of 2500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: MyWebcamFrameStream  with a channel count of 1
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: BalanceBoard_stream  with a channel count of 4
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: Mic  with a channel count of 1
 and a sampling rate of 16000
done with processing a complete time series and audio data
we will now start making trial snipped data
Loading in data from participant: p10
13681.8818345
15.285
time shift calcuated: 15.591900223855191
working on stream: MyWebca

done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: BalanceBoard_stream  with a channel count of 4
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: BrainAmpSeries-Dev_1  with a channel count of 10
 and a sampling rate of 2500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: Mic  with a channel count of 1
 and a sampling rate of 16000
done with processing a complete time series and audio data
we will now start making trial snipped data
Loading in data from participant: p2
5896.275649900001
24.435
time shift calcuated: -8054.411386221593
working on stream: MyWebcamFrameStream  with a channel count of 1
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream

done with processing a complete time series and audio data
we will now start making trial snipped data
working on stream: Mic  with a channel count of 1
 and a sampling rate of 16000
done with processing a complete time series and audio data
we will now start making trial snipped data
Were done: proceed to snipping videos to triallevel


In [6]:
datafolders = ['p15']

# breaking video files into trialsized videos

In [7]:
import cv2
import random
import json
errorlist = []

#read the timeshifts
file_path = file_path = timejson+'timeshift_dictionary.json'
with open(file_path, "r") as file:
    json_string = file.read()
# Parse the JSON string into a dictionary
timeshiftdict = json.loads(json_string)

for dat in datafolders:
    print('Loading in trial data from participant: ' + dat)
    #extract trialdata
    trialdata = pd.read_csv(targetfolder+dat+'/triallist_'+dat+'.csv', sep=";")
    videolong = targetfolder+dat+'/'+dat+'-converted.mp4' #
    wavdata = pd.read_csv(outputfolder+dat+'_'+'Mic'+'_nominal_srate'+str(16000)+'.csv')
    framestream = pd.read_csv(outputfolder+dat+'_'+'MyWebcamFrameStream'+'_nominal_srate'+str(500)+'.csv')
    start = min(wavdata['0'])
    for index, row in trialdata.iterrows():
        begin = row['trialstarted']-timeshiftdict[dat] #begintime trial since buttonbox press
        end = row['trialended']-timeshiftdict[dat] #endtime trial
        trialindex = row['trialindex'] #the trial ID number
        if os.path.exists(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'video_raw'+'.mp4'):
            print('file exists: '+trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'video_raw'+'.mp4')
        if not os.path.exists(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'video_raw'+'.mp4'):
            if not os.path.exists(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'Mic'+'.csv'):
                errorlist.append(trialfolder+dat+'_trial_'+ str(trialindex))
                print('there is an issue with no microphone data for ppn ' + dat + ', trial: ' + str(trialindex))
                break
            if os.path.exists(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'Mic'+'.csv'):
                wavstream = pd.read_csv(trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'Mic'+'.csv')           
                if len(wavstream['1'])==0:
                    errorlist.append(trialfolder+dat+'_trial_'+ str(trialindex))
                    print(trialfolder+dat+'_trial_'+ str(trialindex))
                if len(wavstream['1'])>0:
                    #wavstream['0'] = wavstream['0']-start
                    bg = min(wavstream['0'])
                    en = max(wavstream['0'])
                    vidloc = trialfolder+dat+'_trial_'+ str(trialindex) +'_'+'video_raw'+'.mp4'
                    indices = (framestream.loc[:,'0'] >= begin) & (framestream.loc[:,'0'] <= end) #only select the framestream which includes trialtime
                    sframes = framestream.loc[indices, '1'] #get a framelist that was in trialtime
                    startframe = int(min(sframes)) #get the first frame in trial
                    endframe = int(max(sframes))+1#get the last frame in trial (add 1 so that the range is correct)
                    totframes = endframe-startframe
                    frames = range(startframe, endframe) #get all the frames in trial
                    capture = cv2.VideoCapture(videolong) #load in the videocapture
                    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH) #check frame width
                    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT) #check frame height
                    frate =  capture.get(cv2.CAP_PROP_FPS)
                    fourcc = cv2.VideoWriter_fourcc(*'M','J','P','G') #for different video formats you could use e.g., *'XVID'
                    originalfps = round((totframes/(end-begin)),3)
                    print(totframes)
                    out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
                    frame_count = 0
                    #while capture.isOpened():
                        # Read the next frame
                    #    ret, frame = capture.read()
                    #    if ret:
                            # Increment the frame count
                    #        frame_count += 1
                    #    if frame_count in frames:
                    #        out.write(frame)
                    #    if frame_count > endframe:
                    #        capture.release()
                    #out.release() 

                    for fra in frames:
                        capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
                        ret, frame = capture.read()
                        if ret:
                            out.write(frame)
                        if not ret:
                            print('a frame was dropped: ' + str(fra))
                    capture.release()
                    out.release()              
                if dat == 'p15':
                    bg
                #ffmpeg_extract_subclip(videolong, bg, en, targetname=vidloc) #FLAGGED!
                #Alternative: with VideoFileClip(videolong) as video:
                    #new = video.subclip(bg, en)
                    #new.write_videofile(vidloc, audio_codec='aac')

errors = pd.DataFrame(errorlist, columns=['file_error'])
file_path = errorlogs+'error_log_cuttingvideos_may2023.csv'
df.to_csv(file_path, index=False)          

Loading in trial data from participant: p15
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_1_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_2_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_3_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_4_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_5_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_6_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_7_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_8_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_9_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_10_video_raw.mp4
1
file exists: F:/exp_100_data/Data_processed_2/Data_trials/p15_trial_11_video_raw.mp4
1
file exi

KeyboardInterrupt: 

3939.2157548187056

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


0

# Motion tracking trial-sized videos

In [5]:
#load in required packages
import mediapipe as mp #mediapipe
import cv2 #opencv
import math #basic operations
import numpy as np #basic operations
import pandas as pd #data wrangling
import csv #csv saving
import os #some basic functions for inspecting folder structure etc.

#list all videos in input_videofolder
from os import listdir
from os.path import isfile, join
mypath = datafolder+'Data_processed_2/Data_trials/' #this is your folder with (all) your video(s)
    #files to get
vfiles = glob.glob(mypath +'*.mp4')

#time series output folder
outputf_mask = datafolder+'Data_processed_2/Data_trials/'
outtputf_ts = datafolder+'Data_processed_2/Data_trials/'

#check videos to be processed
print("The following folder is set as the output folder where all the pose time series are stored")
print(os.path.abspath(outtputf_ts))
print("\n The following folder is set as the output folder for saving the masked videos ")
print(os.path.abspath(outputf_mask))
print("\n The following video(s) will be processed for masking: ")
print(vfiles)

#initialize modules and functions

#load in mediapipe modules
mp_holistic = mp.solutions.holistic
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

##################FUNCTIONS AND OTHER VARIABLES
#landmarks 33x that are used by Mediapipe (Blazepose)
markersbody = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_OUTER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
          'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 
          'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
          'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
          'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']

markershands = ['LEFT_WRIST', 'LEFT_THUMB_CMC', 'LEFT_THUMB_MCP', 'LEFT_THUMB_IP', 'LEFT_THUMB_TIP', 'LEFT_INDEX_FINGER_MCP',
              'LEFT_INDEX_FINGER_PIP', 'LEFT_INDEX_FINGER_DIP', 'LEFT_INDEX_FINGER_TIP', 'LEFT_MIDDLE_FINGER_MCP', 
               'LEFT_MIDDLE_FINGER_PIP', 'LEFT_MIDDLE_FINGER_DIP', 'LEFT_MIDDLE_FINGER_TIP', 'LEFT_RING_FINGER_MCP', 
               'LEFT_RING_FINGER_PIP', 'LEFT_RING_FINGER_DIP', 'LEFT_RING_FINGER_TIP', 'LEFT_PINKY_FINGER_MCP', 
               'LEFT_PINKY_FINGER_PIP', 'LEFT_PINKY_FINGER_DIP', 'LEFT_PINKY_FINGER_TIP',
              'RIGHT_WRIST', 'RIGHT_THUMB_CMC', 'RIGHT_THUMB_MCP', 'RIGHT_THUMB_IP', 'RIGHT_THUMB_TIP', 'RIGHT_INDEX_FINGER_MCP',
              'RIGHT_INDEX_FINGER_PIP', 'RIGHT_INDEX_FINGER_DIP', 'RIGHT_INDEX_FINGER_TIP', 'RIGHT_MIDDLE_FINGER_MCP', 
               'RIGHT_MIDDLE_FINGER_PIP', 'RIGHT_MIDDLE_FINGER_DIP', 'RIGHT_MIDDLE_FINGER_TIP', 'RIGHT_RING_FINGER_MCP', 
               'RIGHT_RING_FINGER_PIP', 'RIGHT_RING_FINGER_DIP', 'RIGHT_RING_FINGER_TIP', 'RIGHT_PINKY_FINGER_MCP', 
               'RIGHT_PINKY_FINGER_PIP', 'RIGHT_PINKY_FINGER_DIP', 'RIGHT_PINKY_FINGER_TIP']
facemarks = [str(x) for x in range(478)] #there are 478 points for the face mesh (see google holistic face mesh info for landmarks)

print("Note that we have the following number of pose keypoints for markers body")
print(len(markersbody))

print("\n Note that we have the following number of pose keypoints for markers hands")
print(len(markershands))

print("\n Note that we have the following number of pose keypoints for markers face")
print(len(facemarks ))

#set up the column names and objects for the time series data (add time as the first variable)
markerxyzbody = ['time']
markerxyzhands = ['time']
markerxyzface = ['time']

for mark in markersbody:
    for pos in ['X', 'Y', 'Z', 'visibility']: #for markers of the body you also have a visibility reliability score
        nm = pos + "_" + mark
        markerxyzbody.append(nm)
for mark in markershands:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzhands.append(nm)
for mark in facemarks:
    for pos in ['X', 'Y', 'Z']:
        nm = pos + "_" + mark
        markerxyzface.append(nm)

#check if there are numbers in a string
def num_there(s):
    return any(i.isdigit() for i in s)

#take some google classification object and convert it into a string
def makegoginto_str(gogobj):
    gogobj = str(gogobj).strip("[]")
    gogobj = gogobj.split("\n")
    return(gogobj[:-1]) #ignore last element as this has nothing

#make the stringifyd position traces into clean numerical values
def listpostions(newsamplemarks):
    newsamplemarks = makegoginto_str(newsamplemarks)
    tracking_p = []
    for value in newsamplemarks:
        if num_there(value):
            stripped = value.split(':', 1)[1]
            stripped = stripped.strip() #remove spaces in the string if present
            tracking_p.append(stripped) #add to this list  
    return(tracking_p)

#We will now loop over all the videos that are present in the video file
for vidf in vfiles:
    print("We will now process video:")
    print(vidf)
    print("This is video number" + str(vfiles.index(vidf))+ "of" + str(len(vfiles)) + "videos in total")
    #capture the video, and check video settings
    videoname = vidf[0:(len(vidf)-len('_video_raw.mp4'))]
    videoloc = datafolder+'Data_processed_2/Data_trials/Identifiable/' + vidf
    capture = cv2.VideoCapture(videoloc) #load in the videocapture
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH) #check frame width
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT) #check frame height
    samplerate = capture.get(cv2.CAP_PROP_FPS)
    #read in audio data to get times
    trialdata = pd.read_csv(trialfolder+vidf[0:(len(vidf)-len('_video_raw.mpeg'))] +'_Mic.csv')
    #make an 'empty' video file where we project the pose tracking on
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #for different video formats you could use e.g., *'XVID'
    out = cv2.VideoWriter(outputf_mask+videoname+'_tracked.mp4', fourcc, 
                          fps = samplerate, frameSize = (int(frameWidth), int(frameHeight)))

    # Run MediaPipe frame by frame using Holistic with `enable_segmentation=True` to get pose segmentation.
    time = min(trialdata['0']) #we start the te time based on mic timeseries
    tsbody = [markerxyzbody]   #these will be your time series objects, which start with collumn names initialized above
    tshands = [markerxyzhands] #these will be your time series objects, which start with collumn names initialized above
    tsface = [markerxyzface]   #these will be your time series objects, which start with collumn names initialized above
    with mp_holistic.Holistic(
            static_image_mode=True, enable_segmentation=True, refine_face_landmarks=True) as holistic:
        while (True):
            ret, image = capture.read() #read frame
            if ret == True: #if there is a frame
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #make sure the image is in RGB format
                results = holistic.process(image) #apply Mediapipe holistic processing
                # Draw pose segmentation
                h, w, c = image.shape
                original_image = np.concatenate([image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
                mask_img = np.zeros_like(image, dtype=np.uint8) #set up basic mask image
                if  np.all(results.segmentation_mask) != None: #check if there is a pose found
                    mask_img[:, :] = (255,255,255) #set up basic mask image
                    segm_2class = 0.2 + 0.8 * results.segmentation_mask #set up a segmentation of the results of mediapipe
                    segm_2class = np.repeat(segm_2class[..., np.newaxis], 3, axis=2) #set up a segmentation of the results of mediapipe
                    annotated_image = mask_img * segm_2class * (1 - segm_2class) #take the basic mask image and make a sillhouette mask
                    # append Alpha channel to sillhouetted mask so that we can overlay it to the original image
                    mask = np.concatenate([annotated_image, np.full((h, w, 1), 255, dtype=np.uint8)], axis=-1)
                    # Zero background where we want to overlay
                    original_image[mask==0]=0 #for the original image we are going to set everything at zero for places where the mask has to go
                    original_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)
                    #now lets draw on the original_image the left and right hand landmarks, the facemesh and the body poses
                        #left hand
                    mp_drawing.draw_landmarks(original_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                        #right hand
                    mp_drawing.draw_landmarks(original_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                        #face
                    mp_drawing.draw_landmarks(
                            original_image,
                            results.face_landmarks,
                            mp_holistic.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles
                            .get_default_face_mesh_tesselation_style())
                        #body
                    mp_drawing.draw_landmarks(
                            original_image,
                            results.pose_landmarks,
                            mp_holistic.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp_drawing_styles.
                            get_default_pose_landmarks_style())
                    #######################now save everything to a time series
                    #make a variable list with x, y, z, info where data is appended to
                            #Real-world 3D coordinates in meters with the origin at the center between hips.
                    samplebody = listpostions(results.pose_world_landmarks) #listpostions(results.pose_landmarks)
                    samplehands = listpostions([results.left_hand_landmarks, results.right_hand_landmarks])
                    sampleface = listpostions(results.face_landmarks)
                    samplebody.insert(0, time)
                    samplehands.insert(0, time)
                    sampleface.insert(0, time)
                    tsbody.append(samplebody)   #append to the timeseries object
                    tshands.append(samplehands) #append to the timeseries object
                    tsface.append(sampleface)   #append to the timeseries object
                #show the video as we process (you can comment this out, if you want to run this process in the background)
                cv2.imshow("resizedimage", original_image)
                out.write(original_image) #save the frame to the new masked video
                time = time+(1000/samplerate)#update the time variable  for the next frame
            if cv2.waitKey(1) == 27: #allow the use of ESCAPE to break the loop
                   break
            if ret == False: #if there are no more frames, break the loop
                break

    #once done de-initialize all processes
    out.release()
    capture.release()
    cv2.destroyAllWindows()
    #add audio to videos
    #video_clip = VideoFileClip(outputf_mask+videoname+'_tracked.mp4')
    ## load the audio
    #wavloc = trialfolder+vidf[0:(len(vidf)-len('_video_raw.mpeg'))] +'_Mic.wav'
    #mp3loc = trialfolder+'temp/'+vidf[0:(len(vidf)-len('_video_raw.mpeg'))] +'_Mic.mp3'
    #AudioSegment.from_wav(wavloc).export(mp3loc, format="mp3")    
    ## Create "in-memory" buffer
    #mptoc = AudioFileClip(mp3loc)
    #final_clip = video_clip.set_audio(mptoc)
    #final_clip.write_videofile(outputf_mask+videoname+'_withMic.mp4',codec= 'mpeg4' ,audio_codec='libvorbis')
     ####################################################### data to be written row-wise in csv fil
    # opening the csv file in 'w+' mode
    filebody = open(outtputf_ts + vidf[:-4]+'_body.csv', 'w+', newline ='')
    #write it
    with filebody:    
        write = csv.writer(filebody)
        write.writerows(tsbody)
    # #  opening the csv file in 'w+' mode
    #filehands = open(outtputf_ts + vidf[:-4]+'_hands.csv', 'w+', newline ='')
    ##write it
    #with filehands:
    #    write = csv.writer(filehands)
    #    write.writerows(tshands)
    ## opening the csv file in 'w+' mode
    #fileface = open(outtputf_ts + vidf[:-4]+'_face.csv', 'w+', newline ='')
    ##write it
    #with fileface:    
    #    write = csv.writer(fileface)
    #    write.writerows(tsface)

print("Done with processing all folders; go look in your output folders!")  



The following folder is set as the output folder where all the pose time series are stored
D:\VENI_data_Local\Data_processed_2\Data_trials

 The following folder is set as the output folder for saving the masked videos 
D:\VENI_data_Local\Data_processed_2\Data_trials

 The following video(s) will be processed for masking: 
['pilot3_trial_10_video_raw.mpeg', 'pilot3_trial_11_video_raw.mpeg', 'pilot3_trial_12_video_raw.mpeg', 'pilot3_trial_13_video_raw.mpeg', 'pilot3_trial_14_video_raw.mpeg', 'pilot3_trial_15_video_raw.mpeg', 'pilot3_trial_16_video_raw.mpeg', 'pilot3_trial_17_video_raw.mpeg', 'pilot3_trial_18_video_raw.mpeg', 'pilot3_trial_19_video_raw.mpeg', 'pilot3_trial_1_video_raw.mpeg', 'pilot3_trial_20_video_raw.mpeg', 'pilot3_trial_21_video_raw.mpeg', 'pilot3_trial_22_video_raw.mpeg', 'pilot3_trial_23_video_raw.mpeg', 'pilot3_trial_24_video_raw.mpeg', 'pilot3_trial_25_video_raw.mpeg', 'pilot3_trial_26_video_raw.mpeg', 'pilot3_trial_27_video_raw.mpeg', 'pilot3_trial_28_video_raw.mp

We will now process video:
pilot3_trial_26_video_raw.mpeg
This is video number17of176videos in total
We will now process video:
pilot3_trial_27_video_raw.mpeg
This is video number18of176videos in total
We will now process video:
pilot3_trial_28_video_raw.mpeg
This is video number19of176videos in total
We will now process video:
pilot3_trial_29_video_raw.mpeg
This is video number20of176videos in total
We will now process video:
pilot3_trial_2_video_raw.mpeg
This is video number21of176videos in total
We will now process video:
pilot3_trial_30_video_raw.mpeg
This is video number22of176videos in total
We will now process video:
pilot3_trial_31_video_raw.mpeg
This is video number23of176videos in total
We will now process video:
pilot3_trial_32_video_raw.mpeg
This is video number24of176videos in total
We will now process video:
pilot3_trial_33_video_raw.mpeg
This is video number25of176videos in total
We will now process video:
pilot3_trial_34_video_raw.mpeg
This is video number26of176videos 

We will now process video:
pilot4_trial_20_video_raw.mpeg
This is video number99of176videos in total
We will now process video:
pilot4_trial_21_video_raw.mpeg
This is video number100of176videos in total
We will now process video:
pilot4_trial_22_video_raw.mpeg
This is video number101of176videos in total
We will now process video:
pilot4_trial_23_video_raw.mpeg
This is video number102of176videos in total
We will now process video:
pilot4_trial_24_video_raw.mpeg
This is video number103of176videos in total
We will now process video:
pilot4_trial_25_video_raw.mpeg
This is video number104of176videos in total
We will now process video:
pilot4_trial_26_video_raw.mpeg
This is video number105of176videos in total
We will now process video:
pilot4_trial_27_video_raw.mpeg
This is video number106of176videos in total
We will now process video:
pilot4_trial_28_video_raw.mpeg
This is video number107of176videos in total
We will now process video:
pilot4_trial_29_video_raw.mpeg
This is video number108of

Moviepy - Building video ../../../../VENI_data_Local/Data_processed_2/Data_trials/pilot3_trial_10_withMic.mp4.


AttributeError: 'str' object has no attribute 'write_audiofile'